In [3]:
using CSV, DataFrames, Statistics, FreqTables

In [20]:
a1=DataFrame(CSV.File("./csv/asts.csv"))

,ast,courseid,astname,cont
,String7,Int64,String,Float64
1,dqfe,675,Lab Report 1,0.0175
2,eqqr,104,Lab Report 1,0.0161538
3,purz,104,Lab Report 2,0.0161538
4,zteo,675,Lab Report 2,0.0175
5,knny,832,Essay 1,0.25
6,qrfq,675,Lab Report 3,0.0175
7,qdhd,104,Lab Report 3,0.0161538
8,svah,675,Lab Report 4,0.0175
9,pbif,104,Lab Report 4,0.0161538


In [19]:
m1=DataFrame(CSV.File("./csv/marks.csv"))

,ast,studentID,mark
,String7,Int64,Float64
1,dqfe,369773,6.67
2,dqfe,542150,80.0
3,dqfe,373971,80.0
4,dqfe,658468,26.67
5,dqfe,946387,73.33
6,dqfe,505926,48.87
7,dqfe,127232,0.0
8,dqfe,911293,100.0
9,dqfe,377866,80.0


In [21]:
a2=a1[a1.courseid .== 326,:] # filter for course id

,ast,courseid,astname,cont
,String7,Int64,String,Float64
1,mnll,326,Essay 1,0.25
2,nvxk,326,Final Exam,0.25
3,bemh,326,Final Essay,0.25
4,tcbq,326,Attendance,0.05
5,zijh,326,Reading Assessment Quizzes,0.2


In [29]:
m2=DataFrame()

""
""


In [30]:
for i in 1:length(a2[:,:ast])
m2=vcat(m2,m1[m1.ast .== a2[i,:ast],:])
end

In [33]:
m3=m1[m1.ast .== a2[1,:ast],:]

,ast,studentID,mark
,String7,Int64,Float64
1,mnll,104689,89.0
2,mnll,114185,92.0
3,mnll,116679,90.0
4,mnll,116824,89.0
5,mnll,162638,89.0
6,mnll,205269,92.0
7,mnll,224062,89.0
8,mnll,251559,90.0
9,mnll,252873,90.0


In [36]:
unique(m3,:studentID)

,ast,studentID,mark
,String7,Int64,Float64
1,mnll,104689,89.0
2,mnll,114185,92.0
3,mnll,116679,90.0
4,mnll,116824,89.0
5,mnll,162638,89.0
6,mnll,205269,92.0
7,mnll,224062,89.0
8,mnll,251559,90.0
9,mnll,252873,90.0
